In [5]:
from google.colab import drive

In [6]:
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import tensorflow as tf
import numpy as np
import os
from scipy.io import wavfile
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, BatchNormalization
from tensorflow.keras.models import Model

In [8]:
def load_spectrograms(folder_path):
    spectrograms = []
    file_names = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            img = tf.keras.preprocessing.image.load_img(os.path.join(folder_path, filename), color_mode='grayscale')
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            spectrograms.append(img_array)
            file_names.append(filename)
    return np.array(spectrograms), file_names

**VGG19**

In [12]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.models import Model
import numpy as np

def build_denoising_model(base_model):
    # Add custom layers for denoising after the base model
    x = base_model.output
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    # Create the full model
    denoising_model = Model(inputs=base_model.input, outputs=x)
    return denoising_model


def calculate_snr(original, denoised):
    noise = original - denoised
    signal_power = np.mean(original ** 2)
    noise_power = np.mean(noise ** 2)
    snr = 10 * np.log10(signal_power / noise_power)
    return snr

def denoise_spectrograms(spectrograms):
    # Load VGG19 as the feature extractor
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Build the denoising model
    denoising_model = build_denoising_model(base_model)

    # Assume that we are not training the model, so set trainable to False
    for layer in base_model.layers:
        layer.trainable = False

    # Process and denoise each spectrogram
    denoised_spectrograms = []
    mse_list = []
    snr_list = []

    for spec in spectrograms:
        original_shape = spec.shape[:2]  # Store the original shape
        spec_tensor = tf.convert_to_tensor(spec, dtype=tf.float32)
        spec_rgb = tf.image.grayscale_to_rgb(spec_tensor)

        # Resize and preprocess the spectrogram for VGG19
        spec_resized = tf.image.resize(spec_rgb, (224, 224))
        spec_resized = tf.keras.applications.vgg19.preprocess_input(spec_resized)
        spec_resized = tf.expand_dims(spec_resized, axis=0)

        # Predict the denoised spectrogram
        denoised_spec = denoising_model.predict(spec_resized)

        # Resize back to original dimensions
        denoised_spec_resized = tf.image.resize(denoised_spec[0], original_shape)

        # Convert to numpy array for MSE and SNR calculation
        denoised_spec_array = denoised_spec_resized.numpy()[:,:,0]  # Get the grayscale channel

        denoised_spectrograms.append(denoised_spec_array)

        # Calculate MSE and SNR
        mse = calculate_mse(spec[:,:,0], denoised_spec_array)  # Use grayscale channel for calculation
        snr = calculate_snr(spec[:,:,0], denoised_spec_array)
        mse_list.append(mse)
        snr_list.append(snr)

    return np.array(denoised_spectrograms), mse_list, snr_list

# Example usage
folder_path = '/content/drive/MyDrive/Images'
spectrograms, file_names = load_spectrograms(folder_path)
denoised_spectrograms, mse_list, snr_list = denoise_spectrograms(spectrograms)
print(np.mean(snr_list)*250)

1/1 [==============================] - 1s 675ms/step
2.2523850808890225


Summary:
MSE: The value appears to be on the higher side, which might not be ideal.
SNR: The value is low, suggesting that the noise reduction process might not be very effective.

In [13]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.models import Model
import numpy as np

def build_resnet_denoising_model(base_model):
    # Add custom layers for denoising after the base model
    x = base_model.output
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    # Create the full model
    denoising_model = Model(inputs=base_model.input, outputs=x)
    return denoising_model


def calculate_snr(original, denoised):
    noise = original - denoised
    signal_power = np.mean(original ** 2)
    noise_power = np.mean(noise ** 2)
    snr = 10 * np.log10(signal_power / noise_power)
    return snr

def denoise_spectrograms(spectrograms):
    # Load ResNet50 as the feature extractor
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Build the denoising model
    denoising_model = build_resnet_denoising_model(base_model)

    # Set layers to non-trainable
    for layer in base_model.layers:
        layer.trainable = False

    # Process and denoise each spectrogram
    denoised_spectrograms, mse_list, snr_list = [], [], []

    for spec in spectrograms:
        original_shape = spec.shape[:2]
        spec_tensor = tf.convert_to_tensor(spec, dtype=tf.float32)
        spec_rgb = tf.image.grayscale_to_rgb(spec_tensor)

        # Resize and preprocess the spectrogram
        spec_resized = tf.image.resize(spec_rgb, (224, 224))
        spec_resized = tf.keras.applications.resnet50.preprocess_input(spec_resized)
        spec_resized = tf.expand_dims(spec_resized, axis=0)

        # Predict the denoised spectrogram
        denoised_spec = denoising_model.predict(spec_resized)
        denoised_spec_resized = tf.image.resize(denoised_spec[0], original_shape)

        # Convert to numpy array for calculations
        denoised_spec_array = denoised_spec_resized.numpy()[:,:,0]
        denoised_spectrograms.append(denoised_spec_array)

        # Calculate MSE and SNR
        mse = calculate_mse(spec[:,:,0], denoised_spec_array)
        snr = calculate_snr(spec[:,:,0], denoised_spec_array)
        mse_list.append(mse)
        snr_list.append(snr)

    return np.array(denoised_spectrograms), mse_list, snr_list

# Example usage:
folder_path = '/content/drive/MyDrive/Images'
spectrograms, file_names = load_spectrograms(folder_path)
denoised_spectrograms, mse_list, snr_list = denoise_spectrograms(spectrograms)
print(np.mean(snr_list)*250)

1/1 [==============================] - 0s 195ms/step
6.0277699085418135


In [18]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.models import Model
import numpy as np

def build_inception_denoising_model(base_model):
    # Add custom layers for denoising after the base model
    x = base_model.output
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    # Create the full model
    denoising_model = Model(inputs=base_model.input, outputs=x)
    return denoising_model


def calculate_snr(original, denoised):
    noise = original - denoised
    signal_power = np.mean(original ** 2)
    noise_power = np.mean(noise ** 2)
    snr = 10 * np.log10(signal_power / noise_power)
    return snr

def denoise_spectrograms(spectrograms):
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    denoising_model = build_inception_denoising_model(base_model)

    # Set layers to non-trainable
    for layer in base_model.layers:
        layer.trainable = False

    # Process and denoise each spectrogram
    denoised_spectrograms, mse_list, snr_list = [], [], []

    for spec in spectrograms:
        original_shape = spec.shape[:2]
        spec_tensor = tf.convert_to_tensor(spec, dtype=tf.float32)
        spec_rgb = tf.image.grayscale_to_rgb(spec_tensor)

        # Resize and preprocess the spectrogram
        spec_resized = tf.image.resize(spec_rgb, (224, 224))
        spec_resized = tf.keras.applications.resnet50.preprocess_input(spec_resized)
        spec_resized = tf.expand_dims(spec_resized, axis=0)

        # Predict the denoised spectrogram
        denoised_spec = denoising_model.predict(spec_resized)
        denoised_spec_resized = tf.image.resize(denoised_spec[0], original_shape)

        # Convert to numpy array for calculations
        denoised_spec_array = denoised_spec_resized.numpy()[:,:,0]
        denoised_spectrograms.append(denoised_spec_array)

        # Calculate MSE and SNR
        mse = calculate_mse(spec[:,:,0], denoised_spec_array)
        snr = calculate_snr(spec[:,:,0], denoised_spec_array)
        mse_list.append(mse)
        snr_list.append(snr)

    return np.array(denoised_spectrograms), mse_list, snr_list

# Example usage:
folder_path = '/content/drive/MyDrive/Images'
spectrograms, file_names = load_spectrograms(folder_path)
denoised_spectrograms, mse_list, snr_list = denoise_spectrograms(spectrograms)

print(np.mean(snr_list)*250)

1/1 [==============================] - 0s 147ms/step
3.5470474734514332
